# Generate DS2 Dense TMN (Google Colab)

This notebook sets up a Colab GPU runtime, installs dependencies, mounts Google Drive, and runs the TMN symbol placement to generate DS2 Dense TMN dataset outputs directly in Drive. It does not perform Mask R-CNN training.


In [ ]:
# Colab Runtime and GPU Check
import os, sys, platform, time, json
IN_COLAB = 'google.colab' in sys.modules
print(f"In Colab: {IN_COLAB}")
try:
    import torch
    print(f"Torch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"CUDA device count: {torch.cuda.device_count()}")
        print(f"Current device: {torch.cuda.current_device()}")
        print(f"Device name: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
except Exception as e:
    print('Torch import failed, will install below.', e)

In [ ]:
# Install Dependencies
import sys
def pip_install(pkgs):
    import subprocess
    cmd = [sys.executable, '-m', 'pip', 'install', '-U'] + pkgs
    print('Running:', ' '.join(cmd))
    subprocess.check_call(cmd)
try:
    import torch, torchvision
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    import PIL
    import pycocotools
    print('Dependencies already installed.')
except Exception:
    pkgs = [
        'torch', 'torchvision', 'torchaudio',
        'numpy', 'pandas', 'matplotlib', 'pillow',
        'pycocotools',
        'tqdm',
    ]
    pip_install(pkgs)
    import torch, torchvision
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    import PIL
    import pycocotools
print('torch:', torch.__version__, 'torchvision:', torchvision.__version__)

In [ ]:
# Connect to Google Drive
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_DIR = '/content/drive/MyDrive/omr_copilot'
else:
    import os
    BASE_DIR = os.path.expanduser('~/omr_copilot_colab')
print('BASE_DIR:', BASE_DIR)

In [ ]:
# Environment Variables and Paths
from pathlib import Path
import os
DATA_DIR = Path(BASE_DIR) / 'datasets' / 'ds2_dense_tmn'
OUTPUT_DIR = Path(BASE_DIR) / 'outputs' / 'mask_rcnn'
for p in [DATA_DIR, OUTPUT_DIR]:
    p.mkdir(parents=True, exist_ok=True)
os.environ['OMR_BASE_DIR'] = str(BASE_DIR)
os.environ['OMR_DATA_DIR'] = str(DATA_DIR)
os.environ['OMR_OUTPUT_DIR'] = str(OUTPUT_DIR)
print('DATA_DIR:', DATA_DIR)
print('OUTPUT_DIR:', OUTPUT_DIR)

In [ ]:
# Download or Upload Data
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print('Option A: Upload via files.upload()')
    from google.colab import files
    # Uncomment to use manual upload
    # uploaded = files.upload()
    print('Option B: Use Drive: Place zip at', DATA_DIR)
else:
    print('Running outside Colab; ensure dataset exists at', DATA_DIR)
from pathlib import Path
ZIP_PATH = DATA_DIR / 'ds2_dense_tmn.zip'
EXTRACT_DIR = DATA_DIR
if ZIP_PATH.exists():
    import zipfile
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall(EXTRACT_DIR)
        print('Extracted to', EXTRACT_DIR)
# Verify structure
expected_dirs = ['images', 'segmentation', 'instance', 'jsonlar']
for d in expected_dirs:
    p = EXTRACT_DIR / d
    print(d, 'exists:', p.exists(), 'path:', p)

In [ ]:
# Quick Benchmark: CPU vs GPU
import torch, time
x = torch.randn((2048, 2048))
def bench(device):
    t0 = time.time()
    y = x.to(device) * x.to(device)
    torch.cuda.synchronize() if device.type == 'cuda' else None
    return time.time()-t0
cpu_t = bench(torch.device('cpu'))
gpu_t = None
if torch.cuda.is_available():
    gpu_t = bench(torch.device('cuda'))
print({'cpu_sec': cpu_t, 'gpu_sec': gpu_t})
if torch.cuda.is_available():
    print('GPU mem (allocated):', torch.cuda.memory_allocated())

In [ ]:
# Save Outputs to Drive
import pandas as pd
from pathlib import Path
OUT = Path(OUTPUT_DIR)
OUT.mkdir(parents=True, exist_ok=True)
df = pd.DataFrame({'metric': ['cpu_sec','gpu_sec'], 'value': [0.0, 0.0]})
csv_path = OUT / 'benchmark.csv'
df.to_csv(csv_path, index=False)
print('Wrote', csv_path, 'exists:', csv_path.exists())

In [ ]:
# Clone repo from GitHub (optional if already in Drive)
import os, sys, subprocess, pathlib
REPO_URL = 'https://github.com/mtalhabalci/omr_tmn.git'
WORKDIR = '/content/omr_tmn'
if 'google.colab' in sys.modules:
    if not os.path.isdir(WORKDIR):
        print('Cloning repo...')
        subprocess.check_call(['git','clone',REPO_URL, WORKDIR])
    else:
        print('Repo exists; pulling latest...')
        subprocess.call(['bash','-lc', f'cd {WORKDIR} && git pull --rebase'])
    %cd {WORKDIR}
else:
    print('Not in Colab; skipping clone and cd.')

In [ ]:
# Configure Drive paths for DS2 Dense (source) and DS2 Dense TMN (output)
SRC_ROOT = '/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_dense'
OUT_ROOT = '/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_dense_tmn'
JSON_GLOB = f"{SRC_ROOT}/deepscores_*.json"  # matches train+test
SRC_IMAGES = f"{SRC_ROOT}/images"
print({'SRC_ROOT': SRC_ROOT, 'OUT_ROOT': OUT_ROOT, 'JSON_GLOB': JSON_GLOB})

In [ ]:
# Run TMN placement: from-fs-missing, per-shard JSON, checkpoint flush
import sys, subprocess, os
cmd = [
    sys.executable, 'src/place_tmn_batch.py',
    '--images-dir', SRC_IMAGES,
    '--out-root', OUT_ROOT,
    '--json-glob', JSON_GLOB,
    '--from-fs-missing',
    '--checkpoint', '100',
    '--json-out-mode', 'per-shard',
    '--limit', '0'
]
print('Running:', ' '.join(cmd))
ret = subprocess.call(cmd)
print('Exit code:', ret)

In [ ]:
# Verify outputs
import os, glob, json
img_count = len(glob.glob(f"{OUT_ROOT}/images/*.png"))
seg_count = len(glob.glob(f"{OUT_ROOT}/segmentation/*_seg.png"))
inst_count = len(glob.glob(f"{OUT_ROOT}/instance/*_inst.png"))
json_files = sorted(glob.glob(f"{OUT_ROOT}/jsonlar/*.json"))
print({'images': img_count, 'segmentation': seg_count, 'instance': inst_count, 'jsons': json_files[:5]})

In [ ]:
# Dataset Loader for DS2 Dense TMN (COCO-like)
import json, os, glob
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader

class DS2TMNDataset(Dataset):
    def __init__(self, images_dir, json_paths, transform=None):
        self.images_dir = images_dir
        self.transform = transform
        # merge shards
        self.images = []
        self.ann_by_img = {}
        for jp in json_paths:
            with open(jp,'r',encoding='utf-8') as f:
                data = json.load(f)
            imgs = data.get('images') or []
            anns = data.get('annotations') or {}
            if isinstance(imgs, dict):
                imgs = list(imgs.values())
            for im in imgs:
                fn = im.get('filename') or im.get('file_name')
                if not fn:
                    continue
                self.images.append({'id': int(im.get('id')), 'filename': fn})
            for k,v in anns.items():
                img_id = int(v.get('img_id'))
                self.ann_by_img.setdefault(img_id, []).append(v)
        # deduplicate by filename order
        seen = set()
        uniq = []
        for im in self.images:
            if im['filename'] in seen:
                continue
            seen.add(im['filename'])
            uniq.append(im)
        self.images = uniq

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        im = self.images[idx]
        fn = im['filename']
        path = os.path.join(self.images_dir, fn)
        img = Image.open(path).convert('RGB')
        w, h = img.size
        # build target from annotations (boxes + labels + masks optional later)
        anns = self.ann_by_img.get(im['id'], [])
        boxes = []
        labels = []
        for a in anns:
            b = a.get('a_bbox') or a.get('bbox')
            cats = a.get('cat_id') or []
            if b and len(b)>=4:
                boxes.append([b[0], b[1], b[2], b[3]])
                # single-category per ann
                lab = int(cats[0]) if (isinstance(cats, list) and cats) else 0
                labels.append(lab)
        target = {
            'boxes': torch.tensor(boxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64),
            'image_id': torch.tensor([im['id']])
        }
        if self.transform:
            img = self.transform(img)
        else:
            img = torchvision.transforms.ToTensor()(img)
        return img, target

# Build datasets/loaders
train_jsons = sorted(glob.glob(f"{OUT_ROOT}/jsonlar/*train*.json"))
test_jsons = sorted(glob.glob(f"{OUT_ROOT}/jsonlar/*test*.json"))
print({'train_jsons': train_jsons[:3], 'test_jsons': test_jsons[:3]})
train_ds = DS2TMNDataset(images_dir=SRC_IMAGES, json_paths=train_jsons)
test_ds = DS2TMNDataset(images_dir=SRC_IMAGES, json_paths=test_jsons)

def collate_fn(batch):
    return tuple(zip(*batch))

# Auto batch size based on GPU
import torch
bs = 4
if torch.cuda.is_available():
    name = torch.cuda.get_device_name(0).lower()
    if 'a100' in name or 'l4' in name or 'v100' in name:
        bs = 8
    elif 't4' in name or 'p100' in name:
        bs = 4
train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=bs, shuffle=False, num_workers=2, collate_fn=collate_fn)
print({'batch_size': bs, 'train_len': len(train_ds), 'test_len': len(test_ds)})

In [ ]:
# Mask R-CNN model setup
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

num_classes = 1 + 8  # background + 8 TMN cats
model = maskrcnn_resnet50_fpn(weights=None)
# Replace box head
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
# Replace mask head
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print('Model ready on', device)

In [ ]:
# Training loop with AMP + checkpoints to Drive
import torch, time, os
from torch.optim import SGD
from torch.cuda.amp import GradScaler, autocast

EPOCHS = 3
lr = 0.005
momentum = 0.9
weight_decay = 0.0005
optimizer = SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
scaler = GradScaler(enabled=torch.cuda.is_available())

CKPT_DIR = os.path.join(OUT_ROOT, 'checkpoints')
os.makedirs(CKPT_DIR, exist_ok=True)

model.train()
for epoch in range(EPOCHS):
    t0 = time.time()
    total_loss = 0.0
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=torch.cuda.is_available()):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
        scaler.scale(losses).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += losses.item()
    dur = time.time()-t0
    avg = total_loss / max(1, len(train_loader))
    print({'epoch': epoch+1, 'loss': round(avg,3), 'sec': round(dur,1)})
    # Save checkpoint
    ckpt_path = os.path.join(CKPT_DIR, f'maskrcnn_epoch{epoch+1}.pt')
    torch.save({'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch+1}, ckpt_path)
    print('Saved', ckpt_path)

print('Training complete')

In [ ]:
# Quick eval: run on few test samples and visualize boxes
import matplotlib.pyplot as plt
model.eval()
@torch.no_grad()
def run_eval(n=3):
    cnt = 0
    for images, targets in test_loader:
        images = [img.to(device) for img in images]
        outputs = model(images)
        for img, out in zip(images, outputs):
            if cnt>=n:
                return
            fig, ax = plt.subplots(figsize=(6,6))
            ax.imshow(img.permute(1,2,0).cpu().numpy())
            boxes = out['boxes'].cpu().numpy()
            scores = out['scores'].cpu().numpy()
            for b,s in zip(boxes, scores):
                if s<0.5:
                    continue
                x1,y1,x2,y2 = b
                ax.add_patch(plt.Rectangle((x1,y1), x2-x1, y2-y1, fill=False, color='lime', linewidth=2))
            ax.set_title(f"detections (>=0.5), {len(boxes)} boxes")
            plt.show()
            cnt += 1

run_eval(n=3)

In [ ]:
# Count files under DS2 Complete (images/segmentation/instance + JSON train/test)
import os, glob, json
from pathlib import Path

ROOT = '/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete'
IMAGES_DIR = os.path.join(ROOT, 'images')
SEG_DIR = os.path.join(ROOT, 'segmentation')
INST_DIR = os.path.join(ROOT, 'instance')
JSON_DIR = os.path.join(ROOT, 'jsonlar')

# Collect files
images = sorted(glob.glob(os.path.join(IMAGES_DIR, '*.png')))
segs = sorted(glob.glob(os.path.join(SEG_DIR, '*.png')))
insts = sorted(glob.glob(os.path.join(INST_DIR, '*.png')))
json_all = sorted(glob.glob(os.path.join(JSON_DIR, '*.json')))
json_train = [p for p in json_all if ('train' in os.path.basename(p).lower())]
json_test = [p for p in json_all if ('test' in os.path.basename(p).lower())]

# Derive base names for consistency checks
base_images = {os.path.splitext(os.path.basename(p))[0] for p in images}
base_segs = {os.path.basename(p).replace('_seg.png','') for p in segs}
base_insts = {os.path.basename(p).replace('_inst.png','') for p in insts}

missing_seg = sorted(list(base_images - base_segs))
missing_inst = sorted(list(base_images - base_insts))
extra_seg = sorted(list(base_segs - base_images))
extra_inst = sorted(list(base_insts - base_images))

print({'ROOT': ROOT})
print({'images_png': len(images), 'segmentation_png': len(segs), 'instance_png': len(insts)})
print({'json_total': len(json_all), 'json_train_files': len(json_train), 'json_test_files': len(json_test)})

# Quick consistency report
print({'images_vs_seg_equal': len(base_images)==len(base_segs) and (not missing_seg) and (not extra_seg),
       'images_vs_inst_equal': len(base_images)==len(base_insts) and (not missing_inst) and (not extra_inst)})

# Show small samples of mismatches if any
if missing_seg:
    print('Missing segmentation for first 5 images:', missing_seg[:5])
if missing_inst:
    print('Missing instance for first 5 images:', missing_inst[:5])
if extra_seg:
    print('Segmentation without matching image (first 5):', extra_seg[:5])
if extra_inst:
    print('Instance without matching image (first 5):', extra_inst[:5])
